In [1]:
import random
import cx_Oracle
import datetime 
cx_Oracle.init_oracle_client(lib_dir=r"C:\OracleDBproject\instantclient_21_7")  # 여기서 각자 라이브러리 위치 설정해줘야함
connection = cx_Oracle.connect(user='admin', password='Semorang1234', dsn='semodb_high')
# 본인이 접속할 오라클 클라우드 DB 사용자이름, 비밀번호, dsn을 넣어준다.
print(connection)

<cx_Oracle.Connection to admin@semodb_high>


In [2]:
# INSERT 출력을 위한 OUTPUT_LIST
output_list =[]

In [3]:
# USER 전부 가져오기
cursor = connection.cursor()
cursor.execute("select U.USER_ID from USERS U")
user_rows = cursor.fetchall()
total_user_list = [i[0] for i in user_rows]

In [4]:
# 새로운 영수증 만들고 receipt_id 반환하기
def make_receipt():
    cursor = connection.cursor()
    cursor.execute("select COUNT(*) from RECEIPT")
    receipt_count = cursor.fetchone()[0]
    
    receipt_id = 'receipt_'+ str(receipt_count+1)
    
    year = str(random.randint(2019,2022))
    month = str(random.randint(1,12))
    day = str(random.randint(1,28))
    date = year + '-' + month + '-' + day
    date = datetime.datetime.strptime(date, '%Y-%m-%d')
    
    img_url = 'http://img' + str(receipt_count+1) + '.jpg'

    next_receipt_tuple= (receipt_id, date, img_url)
    cursor.execute("INSERT INTO RECEIPT VALUES (:1,:2,:3)", next_receipt_tuple) 
    print("INSERT INTO RECEIPT VALUES ('{}','{}','{}');".format(next_receipt_tuple[0],next_receipt_tuple[1].date(),next_receipt_tuple[2]))
    return receipt_id

In [5]:
not_verified_thing_query = """WITH NOT_VERIFIED_THING AS
(
	SELECT 
		TEMP.USER_ID AS USER_ID
		,TEMP.THING_ID AS THING_ID
	FROM (
			(SELECT U1.USER_ID, T1.THING_ID
			 FROM THING T1, USERS U1,THINGRANK TR
			 WHERE TR.USER_ID= U1.USER_ID
			 	AND TR.THING_ID = T1.THING_ID)
			
			MINUS
			
			(SELECT U.USER_ID ,T.THING_ID 
			FROM USERS U, THING T ,VERIFY V 
			WHERE U.USER_ID=V.USER_ID
				AND T.THING_ID = V.THING_ID
			)
		) TEMP
	)
SELECT NVT.THING_ID
FROM 
	NOT_VERIFIED_THING NVT
WHERE NVT.USER_ID = :TEMP_USER_ID"""

In [6]:
def random_verify(verify_id,user_id):
    # 해당 유저의 THINGRANK한 THING 가져오기 
    cursor = connection.cursor()
    cursor.execute(not_verified_thing_query,TEMP_USER_ID = user_id)
    user_t_rows = cursor.fetchall()
    if user_t_rows == False:
        return 0
    user_thing_list = [i[0] for i in user_t_rows]

    verify_id = 'verify_'+str(verify_id)
    receipt_id = make_receipt()
    thing_id = random.choice(user_thing_list)
    user_id = user_id
    
    year = str(random.randint(2019,2022))
    month = str(random.randint(1,12))
    day = str(random.randint(1,28))
    verify_date = year + '-' + month + '-' + day
    verify_date = datetime.datetime.strptime(verify_date, '%Y-%m-%d')

    return ((verify_id,receipt_id,thing_id,user_id,verify_date))

In [7]:
# 가져온 THINGRANK에 대해서 VERIFY 진행
def auto_make_verify(n):
    count = 0
    while count < n:
        # user 1명 랜덤 선택
        verify_id = count +1
        user = random.choice(total_user_list)
        one_verify = random_verify(verify_id,user)
        if one_verify == 0:
            continue
        # return ((verify_id,receipt_id,thing_id,user_id,verify_date))
        output_list.append("INSERT INTO VERIFY VALUES ('{}','{}',{},'{}','{}');".format(one_verify[0],one_verify[1],one_verify[2],one_verify[3],one_verify[4].date()))
        cursor.execute("INSERT INTO VERIFY VALUES (:1,:2,:3,:4,:5)", one_verify)        
        count +=1

In [8]:
auto_make_verify(50)

INSERT INTO RECEIPT VALUES ('receipt_1','2020-07-14','http://img1.jpg');
INSERT INTO RECEIPT VALUES ('receipt_2','2022-01-10','http://img2.jpg');
INSERT INTO RECEIPT VALUES ('receipt_3','2021-09-20','http://img3.jpg');
INSERT INTO RECEIPT VALUES ('receipt_4','2022-11-23','http://img4.jpg');
INSERT INTO RECEIPT VALUES ('receipt_5','2021-03-23','http://img5.jpg');
INSERT INTO RECEIPT VALUES ('receipt_6','2019-10-10','http://img6.jpg');
INSERT INTO RECEIPT VALUES ('receipt_7','2021-03-14','http://img7.jpg');
INSERT INTO RECEIPT VALUES ('receipt_8','2020-09-03','http://img8.jpg');
INSERT INTO RECEIPT VALUES ('receipt_9','2021-05-09','http://img9.jpg');
INSERT INTO RECEIPT VALUES ('receipt_10','2021-12-18','http://img10.jpg');
INSERT INTO RECEIPT VALUES ('receipt_11','2019-03-05','http://img11.jpg');
INSERT INTO RECEIPT VALUES ('receipt_12','2020-03-10','http://img12.jpg');
INSERT INTO RECEIPT VALUES ('receipt_13','2021-11-22','http://img13.jpg');
INSERT INTO RECEIPT VALUES ('receipt_14','2

In [9]:
connection.commit()

In [10]:
for o in output_list:
    print(o)

INSERT INTO VERIFY VALUES ('verify_1','receipt_1',16726942,'user_34','2020-12-12');
INSERT INTO VERIFY VALUES ('verify_2','receipt_2',18390076,'user_13','2022-03-19');
INSERT INTO VERIFY VALUES ('verify_3','receipt_3',23317507,'user_50','2020-08-09');
INSERT INTO VERIFY VALUES ('verify_4','receipt_4',26248029,'user_34','2019-01-27');
INSERT INTO VERIFY VALUES ('verify_5','receipt_5',26303950,'user_30','2021-06-15');
INSERT INTO VERIFY VALUES ('verify_6','receipt_6',17720028,'user_39','2022-04-06');
INSERT INTO VERIFY VALUES ('verify_7','receipt_7',18223652,'user_43','2020-04-16');
INSERT INTO VERIFY VALUES ('verify_8','receipt_8',16410914,'user_39','2019-10-10');
INSERT INTO VERIFY VALUES ('verify_9','receipt_9',25458448,'user_32','2022-07-13');
INSERT INTO VERIFY VALUES ('verify_10','receipt_10',25592608,'user_7','2021-01-09');
INSERT INTO VERIFY VALUES ('verify_11','receipt_11',18013943,'user_12','2020-11-06');
INSERT INTO VERIFY VALUES ('verify_12','receipt_12',12074539,'user_34','2